In [ ]:
import os
import re
import time
import json
import pickle

import numpy as np
import mxnet as mx
from mxnet import gluon, autograd, recordio

In [ ]:
class CustomRecordDataset(gluon.data.dataset.RecordFileDataset):
        """A custom dataset wrapping over a RecordIO file containining a feature representation 
        of sample data and its corresponding label.

        Parameters
        ----------
        filename : str
            Path to rec file.
        """

        def __init__(self, filename):
            super(CustomRecordDataset, self).__init__(filename)

        def __getitem__(self, idx):
            record = super(CustomRecordDataset, self).__getitem__(idx)
            header, data = recordio.unpack(record)
            data = pickle.loads(data)
            return mx.nd.array(data), np.array(header.label, dtype=np.int32)

In [ ]:
# Constants
BATCH_SIZE = 128
EPOCHS = 512
NUM_WORKERS = 8

#for randomness
mx.random.seed(999)

In [ ]:
with open('output_acr_label_to_name.json', 'r') as f:
    label_to_name = json.load(f)

num_classes = len(label_to_name)
print(num_classes)

In [ ]:
# Function to define neural network
def custom_model():
    net = gluon.nn.HybridSequential()
    with net.name_scope():
        net.add(gluon.nn.Dropout(.2))
        net.add(gluon.nn.Dense(2000, activation='relu'))
        net.add(gluon.nn.Dropout(.5))
        net.add(gluon.nn.Dense(1000, activation='relu'))
        net.add(gluon.nn.Dropout(.5))
        net.add(gluon.nn.Dense(1000, activation='relu'))
        net.add(gluon.nn.Dropout(.5))
        net.add(gluon.nn.Dense(1000, activation='relu'))
        net.add(gluon.nn.Dropout(.5))
        net.add(gluon.nn.Dense(1000, activation='relu'))
        net.add(gluon.nn.Dropout(.5))
        net.add(gluon.nn.Dense(1000, activation='relu'))
        net.add(gluon.nn.Dropout(.5))
        net.add(gluon.nn.Dense(1000, activation='relu'))
        net.add(gluon.nn.Dropout(.5))
        net.add(gluon.nn.Dense(500, activation='relu'))
        net.add(gluon.nn.Dropout(.5))
        net.add(gluon.nn.Dense(num_classes))
    return net

In [ ]:
# Dataloader for training and validation

train_datapath = 'output_acr_train.rec'
train_dataset = CustomRecordDataset(train_datapath)
train_dataloader = mx.gluon.data.DataLoader(train_dataset, batch_size= BATCH_SIZE,
                                                    num_workers= NUM_WORKERS, shuffle=True)
val_datapath = 'output_acr_val.rec'
val_dataset = CustomRecordDataset(val_datapath)
val_dataloader = mx.gluon.data.DataLoader(val_dataset, batch_size=BATCH_SIZE,
                                                  num_workers=NUM_WORKERS, shuffle=False)


In [ ]:
def acc(output, label):
    # output: (batch, num_classes) float32 ndarray
    # label: (batch, ) int32 ndarray
    return (output.argmax(axis=1)== label.astype('float32')).mean().asscalar()

In [ ]:
if mx.context.num_gpus() > 0:
    print("Running the script on single GPU")
    ctx = mx.gpu(0)
else:
    print("Running the script on CPU")
    ctx = mx.cpu()

In [ ]:
# Create a model
net = custom_model()
net.cast('float32')
net.hybridize(static_alloc=True, static_shape=True)

In [ ]:
# Initialize parameters
initializer = mx.init.Xavier(rnd_type='gaussian', factor_type="in",
                             magnitude=2)
net.initialize(initializer, ctx=ctx)

# Create optimizer
optimizer_params = {'learning_rate': 0.0005}
opt = mx.optimizer.create('Adam', **optimizer_params)

trainer = gluon.Trainer(net.collect_params(), opt)
loss_fn = gluon.loss.SoftmaxCrossEntropyLoss()

In [ ]:
# Function to train the model
def train(net, train_dataloader, val_dataloader):

    best_acc = 0.0
    tic = time.time()
    
    for epoch in range(EPOCHS):
        print('Epoch {}/{}'.format(epoch+1, EPOCHS))
        print('-' * 10)

        train_loss, train_acc, val_acc = 0., 0., 0.
        
        for data, label in train_dataloader:
            data = data.as_in_context(ctx)
            label = label.as_in_context(ctx)
            
            with autograd.record():
                output = net(data)
                L = loss_fn(output, label)
            L.backward()

            trainer.step(BATCH_SIZE)
 
            train_loss += L.mean().asscalar()
            train_acc += acc(output, label)

        epoch_loss = train_loss / len(train_dataloader)
        epoch_acc = train_acc/ len(train_dataloader)
        print('Train loss: {:.4f} accuracy: {:.4f}'.format(epoch_loss, epoch_acc))    
        
        
        for data, label in val_dataloader:
            data = data.as_in_context(ctx)
            label = label.as_in_context(ctx)
            val_acc += acc(net(data), label)
            
        epoch_acc_val = val_acc/ len(val_dataloader)
        print('Validation accuracy: {:.4f}'.format(epoch_acc_val))
        print()
        if epoch_acc_val > best_acc:
            best_acc = epoch_acc_val
            
    print()    
    time_elapsed = time.time() - tic
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best validation accuracy: {:4f}'.format(best_acc))
         
            


In [ ]:
train(net, train_dataloader, val_dataloader)